In [83]:
import numpy as np
import pandas as pd
import json

from pathlib import Path  


In [84]:
data = []

In [ ]:
with open('./DATA/all_data.json') as d:
    data = json.load(d)
    
rows = []
subj = []
keywords = []
affiliations = []
funds = []

for record in data:
    try:
        title = record['abstracts-retrieval-response']['coredata']['dc:title']
        publish_year = record['abstracts-retrieval-response']['item']['ait:process-info']['ait:date-sort']['@year']
        eid = record['abstracts-retrieval-response']['coredata']['eid']
        aggregation_type = record['abstracts-retrieval-response']['coredata']['prism:aggregationType']
        authors = record['abstracts-retrieval-response']['authors']['author']
        author_count = len(authors)
        if record['abstracts-retrieval-response']['language']:
            language = record['abstracts-retrieval-response']['language']['@xml:lang']
        else: language = None
        
        rows.append({
            'Title': title,
            'Publish_year': publish_year,
            'Eid': eid,
            'Aggregation_type': aggregation_type,
            'Author_Count': author_count,
            'Language' : language
        })
        if "subject-areas" in record['abstracts-retrieval-response']:
            areas = record['abstracts-retrieval-response']['subject-areas']['subject-area']
            for area in areas:
                subj.append({
                    'Eid': eid,
                    'Subject_Area': area["$"] if "$" in area else None,
                    'Subject_Code': area["@code"] if "@code" in area else None,
                })
        
        if "author-keywords" in record['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-info']:
            keyword = record['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-info']['author-keywords']['author-keyword']
            for key in keyword:
                keywords.append({
                    'Eid':eid,
                    'Keyword': key["$"] if "$" in key else None,
                    'Language':key["@xml:lang"] if "@xml:lang" in key else None,
                    'Original':key["@original"] if "@original" in key else None
                })
            
        if "affiliation" in record['abstracts-retrieval-response']:
            affils = record['abstracts-retrieval-response']['affiliation']
            for affil in affils:
                affiliations.append({
                    'Eid': eid,
                    'Affil_Id':affil["@id"] if "@id" in affil else None,
                    'Affil_Name':affil["affilname"] if "affilname" in affil else None,
                    'Affil_Country':affil["affiliation-country"] if "affiliation-country" in affil else None,
                    'Affil_City':affil["affiliation-city"] if "affiliation-city" in affil else None
                })
        
        if "@has-funding-info" in record['abstracts-retrieval-response']['item']['xocs:meta']['xocs:funding-list']:
            if "xocs:funding" in record['abstracts-retrieval-response']['item']['xocs:meta']['xocs:funding-list']:
                fundings = record['abstracts-retrieval-response']['item']['xocs:meta']['xocs:funding-list']['xocs:funding']
                for funding in fundings:
                    funds.append({
                        'Eid' : eid,
                        'agency-matched-string' : funding["xocs:funding-agency-matched-string"] if "xocs:funding-agency-matched-string" in funding else None,
                        'agency-acronym': funding["xocs:funding-agency-acronym"] if "xocs:funding-agency-acronym" in funding else None,
                        'agency': funding["xocs:funding-agency"] if "xocs:funding-agency" in funding else None,
                        'agency-id': funding["xocs:funding-agency-id"] if "xocs:funding-agency-id" in funding else None,
                        'agency-country': funding["xocs:funding-agency-country"] if "xocs:funding-agency-country" in funding else None
                    })
            else:
                funds.append({
                    'Eid' : eid,
                    'agency-matched-string' : None,
                    'agency-acronym': None,
                    'agency': None,
                    'agency-id': None,
                    'agency-country': None
                })
    except:
        pass

df = pd.DataFrame(rows)

subject_df = pd.DataFrame(subj)

keyword_df = pd.DataFrame(keywords)

affil_df = pd.DataFrame(affiliations)

funds_df = pd.DataFrame(funds)

In [86]:
df = df.drop_duplicates()
subject_df = subject_df.drop_duplicates()
keyword_df = keyword_df.drop_duplicates()
affil_df = affil_df.drop_duplicates()
funds_df = funds_df.drop_duplicates()

In [87]:
df

,Title,Publish_year,Eid,Aggregation_type,Author_Count,Language
0,Effects of iron content on the microstructure ...,2018,2-s2.0-85053164279,Journal,7,eng
1,The critical factors of research and innovatio...,2018,2-s2.0-85049101440,Journal,3,eng
2,Is the occiput-wall distance valid and reliabl...,2018,2-s2.0-85054140369,Journal,8,eng
3,Comparison of soil composition between farmlan...,2018,2-s2.0-85097515350,Journal,2,eng
4,The impact of wire caliber on ERCP outcomes: a...,2018,2-s2.0-85041527766,Journal,13,eng
...,...,...,...,...,...,...
20211,A Techno-Economic Assessment of a Second-Life ...,2023,2-s2.0-85152540548,Journal,3,eng
20212,Encouraging green product purchase: Green valu...,2023,2-s2.0-85132634561,Journal,2,eng
20213,Does leukocytosis remain a predictive factor f...,2023,2-s2.0-85150789915,Journal,16,eng
20214,Administration of ketoprofen in postpartum sow...,2023,2-s2.0-85165609857,Journal,8,eng


In [88]:
df.isnull().sum()

Title                 0
Publish_year          0
Eid                   0
Aggregation_type      0
Author_Count          0
Language            120
dtype: int64

In [89]:
subject_df

""


In [90]:
subject_df.isnull().sum()

Series([], dtype: float64)

In [91]:
keyword_df

""


In [92]:
keyword_df.isnull().sum()

Series([], dtype: float64)

In [93]:
affil_df

""


In [94]:
affil_df.isnull().sum()

Series([], dtype: float64)

In [95]:
funds_df

""


In [96]:
funds_df.isnull().sum()

Series([], dtype: float64)

In [97]:
funds_df['agency-matched-string'].value_counts()

KeyError: 'agency-matched-string'

In [ ]:
funds_df['agency-acronym'].value_counts()

agency-acronym
CU        2461
NRCT       832
TRF        683
สวทช       433
TSRI       368
          ... 
ESTRO        1
OUI          1
NAWA         1
HRBUST       1
SNRU         1
Name: count, Length: 1234, dtype: int64

In [ ]:
# Find rows where all fields except 'Eid' are None
filtered_df = funds_df[funds_df.drop(columns=['Eid']).isna().all(axis=1)]

filtered_df


,Eid,agency-matched-string,agency-acronym,agency,agency-id,agency-country
377,2-s2.0-85064067855,None,None,None,None,None
391,2-s2.0-85090523272,None,None,None,None,None
395,2-s2.0-85086174365,None,None,None,None,None
500,2-s2.0-85089601418,None,None,None,None,None
1128,2-s2.0-85079787046,None,None,None,None,None
...,...,...,...,...,...,...
30329,2-s2.0-85167822123,None,None,None,None,None
30339,2-s2.0-85133588862,None,None,None,None,None
30394,2-s2.0-85162092675,None,None,None,None,None
30426,2-s2.0-85151554103,None,None,None,None,None


In [ ]:
subject_df['Subject_Area'].value_counts()

Subject_Area
Multidisciplinary             1088
Materials Science (all)        907
Chemistry (all)                905
Chemical Engineering (all)     755
Infectious Diseases            753
                              ... 
Museology                        1
Emergency Nursing                1
Chiropractics                    1
Optometry                        1
Family Practice                  1
Name: count, Length: 321, dtype: int64

In [ ]:
subject_df['Subject_Code'].value_counts()

Subject_Code
1000    1088
2500     907
1600     905
1500     755
2725     753
        ... 
1209       1
2907       1
3602       1
3610       1
2714       1
Name: count, Length: 321, dtype: int64

In [ ]:
merge_table = df.merge(subject_df,on='Eid',how='left')
merge_table

,Title,Publish_year,Eid,Aggregation_type,Author_Count,Language,Subject_Area,Subject_Code
0,Effects of iron content on the microstructure ...,2018,2-s2.0-85053164279,Journal,7,eng,Materials Science (all),2500
1,Effects of iron content on the microstructure ...,2018,2-s2.0-85053164279,Journal,7,eng,Condensed Matter Physics,3104
2,The critical factors of research and innovatio...,2018,2-s2.0-85049101440,Journal,3,eng,Business and International Management,1403
3,The critical factors of research and innovatio...,2018,2-s2.0-85049101440,Journal,3,eng,"Economics, Econometrics and Finance (all)",2000
4,Is the occiput-wall distance valid and reliabl...,2018,2-s2.0-85054140369,Journal,8,eng,"Physical Therapy, Sports Therapy and Rehabilit...",3612
...,...,...,...,...,...,...,...,...
50059,Administration of ketoprofen in postpartum sow...,2023,2-s2.0-85165609857,Journal,8,eng,Genetics,1311
50060,Administration of ketoprofen in postpartum sow...,2023,2-s2.0-85165609857,Journal,8,eng,Veterinary (all),3400
50061,Role of Anxiety in Willingness to Communicate ...,2023,2-s2.0-85164405594,Journal,3,eng,Language and Linguistics,1203
50062,Role of Anxiety in Willingness to Communicate ...,2023,2-s2.0-85164405594,Journal,3,eng,Linguistics and Language,3310


In [ ]:
df.isnull().sum()

Title                 0
Publish_year          0
Eid                   0
Aggregation_type      0
Author_Count          0
Language            120
dtype: int64

In [ ]:
filepath = Path('DATA/main.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False) 

In [ ]:
filepath = Path('DATA/subject.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
subject_df.to_csv(filepath, index=False) 

In [ ]:
filepath = Path('DATA/keyword.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
keyword_df.to_csv(filepath, index=False) 

In [ ]:
filepath = Path('DATA/funding.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
funds_df.to_csv(filepath, index=False) 